In [1]:
import os
import xmltodict

xml_files_path = "/Users/sathishkumar/AllFiles/miami/XMLify/dydy"
# xml_files = os.listdir(xml_files_path)
# print([x for x in xml_files if x.endswith(".XML")])

xml_files = ['wf_PLD_KYC2_UDM_STG_LOAD.XML', 'WF_STATIC_ACTZ.XML', 'WF_TRAN_STG.XML', 'WF_STATIC_STG.XML', 'WF_TRAN_ACTZ.XML', 'WF_TRAN_PLD.XML', 'WF_STATIC_PLD.XML'] 

#  function to read the xml file and return the json
def read_xml_file(xml_file_path):
    with open(xml_file_path, "r") as f:
        xml_data = f.read()
    # convert the xml data to json
    json_data = xmltodict.parse(xml_data)
    return json_data

def process_xml_file(xml_file):
    xml_file_path = os.path.join(xml_files_path, xml_file)
    json_data = read_xml_file(xml_file_path)
    return(json_data)

wf_pld_kyc2_udm_stg_load_json = process_xml_file('wf_PLD_KYC2_UDM_STG_LOAD.XML')
wf_static_actz_json = process_xml_file('WF_STATIC_ACTZ.XML')
wf_tran_stg_json = process_xml_file('WF_TRAN_STG.XML')
wf_static_stg_json = process_xml_file('WF_STATIC_STG.XML')
wf_tran_actz_json = process_xml_file('WF_TRAN_ACTZ.XML')
wf_tran_pld_json = process_xml_file('WF_TRAN_PLD.XML')
wf_static_pld_json = process_xml_file('WF_STATIC_PLD.XML')


In [10]:
# Test the enhanced filter logic extraction
import importlib
importlib.reload(logic_conn_csv_export)
from logic_conn_csv_export import InformaticaMappingExtractor

# Create a new extractor instance with enhanced logic
extractor_v2 = InformaticaMappingExtractor()

# Test with a specific file that should have filter conditions
print("Testing enhanced filter logic extraction...")
connectors_v2 = extractor_v2.extract_mappings_from_dict(wf_tran_stg_json, 'enhanced_output_v2.csv')

# Look for filter transformations specifically
filter_connectors = [c for c in connectors_v2 if 'filter' in c.to_instance_type.lower() or 'filter' in c.from_instance_type.lower()]
print(f"\nFound {len(filter_connectors)} filter-related connectors")

if filter_connectors:
    print("\nSample filter transformations with logic:")
    for i, connector in enumerate(filter_connectors[:5]):  # Show first 5
        print(f"\n{i+1}. {connector.to_instance} ({connector.to_instance_type})")
        print(f"   Logic: {connector.transformation_logic}")
        print(f"   Order: {connector.transformation_order}")


Testing enhanced filter logic extraction...
Found 34 mapping(s)
Processing Mapping 1: m_SNA_SRC_STG_LOAN_TRANS
  Found 71 connector(s)
  Created lineage graph with 7 nodes and 6 edges
  Transformation orders: {'FF_DUMMY': 1, 'SQ_FF_DUMMY': 2, 'exp_sna_CIS': 3, 'jtx_sna_CIS': 4, 'exp_Data': 5, 'STG_SNA_LOAN_TRANS': 6, 'ff_sna_stg_loan': 6}
Processing Mapping 2: m_SNA_AUDIT_TRANSACTIONS_STAGE
  Found 15 connector(s)
  Created lineage graph with 5 nodes and 4 edges
  Transformation orders: {'STG_SNA_LOAN_TRANS': 1, 'SQ_STG_SNA_LOAN_TRANS': 2, 'exp_SRC_STG_AUDIT': 3, 'ftr_AUDIT': 4, 'Shortcut_to_AUDIT_COUNTS1': 5}
Processing Mapping 3: m_REV_FIS_SRC_STG_FIS_REV_MNTRY_TRAN
  Found 97 connector(s)
  Created lineage graph with 5 nodes and 5 edges
  Transformation orders: {'Shortcut_to_FISTRANS_NON_DELIMITED_DATA_MTF5': 1, 'AMGDSQ_Shortcut_to_FISTRANS_NON_DELIMITED_DATA_MTF5': 2, 'EXPTRANS1': 3, 'FLR_ZERO_AMOUNTS': 4, 'Shortcut_to_STG_FIS_REV_MNTRY_TRAN': 5}
Processing Mapping 4: m_REV_FIS_AUD

In [11]:
# Debug: Inspect the transformation logic cache for filter transformations
print("=== TRANSFORMATION LOGIC CACHE DEBUG ===")
for mapping_name, transformations in extractor_v2.transformation_logic_cache.items():
    print(f"\nMapping: {mapping_name}")
    for trans_name, trans_info in transformations.items():
        if 'filter' in trans_info['type'].lower():
            print(f"  Filter Transformation: {trans_name}")
            print(f"    Type: {trans_info['type']}")
            print(f"    Logic: {trans_info['logic']}")
            print("    ---")

# Also show some other transformation types for comparison
print("\n=== OTHER TRANSFORMATION TYPES (SAMPLE) ===")
count = 0
for mapping_name, transformations in extractor_v2.transformation_logic_cache.items():
    if count >= 3:  # Show only first 3 mappings
        break
    print(f"\nMapping: {mapping_name}")
    for trans_name, trans_info in transformations.items():
        if 'expression' in trans_info['type'].lower():
            print(f"  Expression: {trans_name} -> {trans_info['logic'][:80]}...")
            break
    count += 1


=== TRANSFORMATION LOGIC CACHE DEBUG ===

Mapping: m_SNA_SRC_STG_LOAN_TRANS

Mapping: m_SNA_AUDIT_TRANSACTIONS_STAGE
  Filter Transformation: ftr_AUDIT
    Type: Filter
    Logic: Filter: TRUE

--SRC_CNT>0
    ---

Mapping: m_REV_FIS_SRC_STG_FIS_REV_MNTRY_TRAN
  Filter Transformation: FLR_ZERO_AMOUNTS
    Type: Filter
    Logic: Filter: (DRAW_AMT_MTF5<>0 OR PRN_AMT_MTF5<>0 OR INT_AMT_MTF5 <> 0 OR LATE_FEE_MTF5 <> 0 OR ANNUAL_FEE_MTF5<>0 OR OTHER_FEE_MTF5 <> 0) AND TP_APP_MTF5='003'
    ---

Mapping: m_REV_FIS_AUDIT_TRANS_SRC_STG_MISSED
  Filter Transformation: FLR_ZERO_AMOUNTS
    Type: Filter
    Logic: Filter: DRAW_AMT_MTF5=0 AND PRN_AMT_MTF5=0 AND INT_AMT_MTF5 = 0 AND LATE_FEE_MTF5 = 0 AND ANNUAL_FEE_MTF5=0 AND OTHER_FEE_MTF5 = 0 AND TP_APP_MTF5='003' 
--TP_APP_MTF5<>'001' AND TP_APP_MTF5<>'003'
    ---

Mapping: m_REV_FIS_AUDIT_TRANS_SRC_CNT

Mapping: m_FIS_SRC_STG_FIS_MNTRY_TRAN
  Filter Transformation: FLR_ZERO_AMOUNTS
    Type: Filter
    Logic: Filter: (PRN_AMT_MTF5<>0 OR INT_A